### Natural Processing Language UFPB - [2022.1] <br> 

#### Pratice - Naive Bayes for Sentiment Analyzer  

***


**[*] Description**

The objective of this exercise is to train and evaluate a Naive Bayes classifier to a sentiment analysis task. <br> <br> Text used in training and testing must be preprocessed to include the prefix “NAO_” according to the following rule: <br> <br> The prefix “NAO_” must be added to each word after a negation token (no, nor, never, never, either) until next time punctuation. Scores are considered: **. , ? ! ;** <br> <br>  The training should be performed with 80% of the data and the test with 20%. You must calculate the accuracy of the classifier applied to the test data and compare with the result of Naive Bayes without using the prefix “NAO_”.

#### [#] Dependencies

In [181]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#### [*] Pre-processing

In [170]:
df = pd.read_csv('B2W-Reviews01.csv', low_memory=False)

In [171]:
df.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


In [172]:
# Dropping NULL values

df.dropna(inplace=True, subset=['review_text'])

In [173]:
df['review_text']

0         Estou contente com a compra entrega rápida o ú...
1         Por apenas R$1994.20,eu consegui comprar esse ...
2         SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3         MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4         A entrega foi no prazo, as americanas estão de...
                                ...                        
132368    Vale muito, estou usando no controle do Xbox e...
132369    Prático e barato, super indico o produto para ...
132370    Chegou antes do prazo previsto e corresponde a...
132371    Material fraco, poderia ser melhor. Ficou deve...
132372    Comprei esse produto, quando chegou estava com...
Name: review_text, Length: 129098, dtype: object

In [174]:
df['label'] = df['overall_rating'].apply(lambda x: 'pos' if x >= 3 else 'neg')

In [175]:
df_no = df.copy()
df_no['label'] = df['overall_rating'].apply(lambda x: 'pos' if x >= 3 else 'neg')

In [176]:
def AddTag(data, tokens):
    split = [i.strip() for i in re.split("[,.?!;]", data)]
    bool = False
    
    for i, text in enumerate(split):
        for j in text.split():
            
            if j in tokens:
                bool = True
                continue
            if bool:
                text = text.replace(f' {j}', f" NAO_{j}")
            
        bool = False
        split[i] = text 
    
    return ', '.join(split)

In [177]:
# Considerando possíveis variações de negação
tokens = ['nao', 'nem', 'nunca', 'jamais', 'tampouco', 'não', 'NAO', 'NÃO', 'Não', 'Nao']

In [178]:
df_no['review_text'].head(20)

0     Estou contente com a compra entrega rápida o ú...
1     Por apenas R$1994.20,eu consegui comprar esse ...
2     SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3     MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4     A entrega foi no prazo, as americanas estão de...
5     Excelente produto, por fora em material acríli...
6     produto mto bom, com essa garrafinha vc pode a...
7     Produto excelente qualidade boa câmera desenvo...
8     O barulho e minimo e o vento é bem forte na ve...
9     MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ES...
10    Comprei um pra cara membro da família nesse na...
11    Produto maravilhoso! Não é barulhento, fácil m...
12    Jogo de panelas excelente material, entrega a ...
13    Esse celular não vale nada a bateria não vale ...
14    produto de acordo com o que eu esperava. Somen...
15    a mochila nao esta fechando direito por isso n...
16    Produto condizente com o anúncio, com material...
17    nao tenho nada a reclamar, bom atendimento

In [179]:
df_no['review_text_tag'] = df_no['review_text'].apply(lambda x: AddTag(x, tokens))

In [180]:
# Leve bug quando encontra NAO, repetindo duas vezes

df_no['review_text_tag'].head(20)

0     Estou contente com a compra entrega rápida o ú...
1     Por apenas R$1994, 20, eu consegui comprar ess...
2     SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3     MEU FILHO AMOU, PARECE DE VERDADE COM TANTOS D...
4     A entrega foi no prazo, as americanas estão de...
5     Excelente produto, por fora em material acríli...
6     produto mto bom, com essa garrafinha vc pode a...
7     Produto excelente qualidade boa câmera desenvo...
8     O barulho e minimo e o vento é bem forte na ve...
9     MEU PRODUTO NAO_NAO NAO_NAO_FOI NAO_NAO_ENTREG...
10    Comprei um pra cara membro da família nesse na...
11    Produto maravilhoso, Não NAO_é NAO_barulhento,...
12    Jogo de panelas excelente material, entrega a ...
13    Esse celular não NAO_vale NAO_nada NAO_a NAO_b...
14    produto de acordo com o que eu esperava, Somen...
15    a mochila nao NAO_esta NAO_fechando NAO_direit...
16    Produto condizente com o anúncio, com material...
17    nao NAO_tenho NAO_nada NAO_a NAO_reclamar,

#### [*] Vectorizing and Training

In [186]:
vec = TfidfVectorizer()
vec_no = TfidfVectorizer()

r_vec = vec.fit_transform(df['review_text'])
r_vec_no = vec_no.fit_transform(df_no['review_text_tag'])

In [187]:
X_train, X_test, y_train, y_test = train_test_split(r_vec, df['label'], test_size=0.2)

X_train_, X_test_, y_train_, y_test_ = train_test_split(r_vec_no, df_no['label'], test_size=0.2, random_state=42)

In [188]:
nb = MultinomialNB()
nb_no = MultinomialNB()

nb.fit(X_train, y_train)
nb_no.fit(X_train_, y_train_)

MultinomialNB()

In [189]:
print(classification_report(y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

         neg       0.86      0.70      0.77      6774
         pos       0.90      0.96      0.93     19046

    accuracy                           0.89     25820
   macro avg       0.88      0.83      0.85     25820
weighted avg       0.89      0.89      0.89     25820

